In [ ]:
!pip3 install ctransformers #transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 38.5 MB/s eta 0:00:00


In [ ]:
! nvidia-smi

Wed May 15 03:07:37 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!pip3 install tqdm pandas

In [ ]:
! pip install gdown

In [49]:
import pandas as pd
from tqdm import tqdm


In [ ]:

from ctransformers import AutoModelForCausalLM
# from transformers import AutoTokenizer, AutoModelForCausalLM

# tokenizer = AutoTokenizer.from_pretrained("gpt2")
# model = AutoModelForCausalLM.from_pretrained("gpt2")

llm = AutoModelForCausalLM.from_pretrained("TheBloke/Llama-2-7B-Chat-GGUF",
                                           # Small model 2.83 GB: llama-2-7b-chat.Q2_K.gguf
                                           # Medium 3.60 GB: llama-2-7b-chat.Q3_K_L.gguf
                                           # Large 3.86 GB: llama-2-7b-chat.Q4_K_S.gguf
                                           # X-Large 4.08 GB: llama-2-7b-chat.Q4_K_M.gguf
                                          model_file="llama-2-7b-chat.Q4_K_M.gguf",
                                          gpu_layers=50)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

llama-2-7b-chat.Q4_K_M.gguf:   0%|          | 0.00/4.08G [00:00<?, ?B/s]

In [ ]:
llm

In [ ]:
! gdown --id 1okDkwGEkiYvz7F67-l1iczmziYQdTBCT

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:138: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1okDkwGEkiYvz7F67-l1iczmziYQdTBCT
To: /content/csvs.tar.gz
100% 1.18M/1.18M [00:00<00:00, 118MB/s]


In [ ]:
! tar -xvf csvs.tar.gz

train_MK_FuturePhysics_Book.csv
train_MK_Physics_of_The_Mind_Book.csv
train_MK_Physics_of_The_Impossible_Book.csv
train_MK_Hyperspace_Book.csv


In [ ]:
all_csvs = [
    "data/train_MK_FuturePhysics_Book.csv",
    "data/train_MK_Hyperspace_Book.csv",
    "data/train_MK_Physics_of_The_Impossible_Book.csv",
    "data/train_MK_Physics_of_The_Mind_Book.csv"
]

In [53]:
all_responsez = open("data/all_responsez.txt", 'a')

In [ ]:
! ls -lh


total 4.5M
-rw-r--r-- 1 root root    0 May 15 03:44 all_responsez.txt
-rw-r--r-- 1 root root 1.2M May 15 03:43 csvs.tar.gz
drwxr-xr-x 1 root root 4.0K Feb  5 20:33 sample_data
-rw-rw-r-- 1 1000 1000 967K May 14 10:29 train_MK_FuturePhysics_Book.csv
-rw-rw-r-- 1 1000 1000 826K May 14 10:33 train_MK_Hyperspace_Book.csv
-rw-rw-r-- 1 1000 1000 771K May 14 10:31 train_MK_Physics_of_The_Impossible_Book.csv
-rw-rw-r-- 1 1000 1000 818K May 14 10:36 train_MK_Physics_of_The_Mind_Book.csv


In [56]:
for csvf in all_csvs:
    text_chunks = pd.read_csv(csvf)
    print("text_chunks: ", len(text_chunks))

    if csvf == "data/train_MK_FuturePhysics_Book.csv":
        sampled_chunks = text_chunks['text'].sample(100, replace=False)
    elif csvf == "data/train_MK_Hyperspace_Book.csv":
        sampled_chunks = text_chunks['text']
    elif csvf == "data/train_MK_Physics_of_The_Impossible_Book.csv":
        sampled_chunks = text_chunks['text'].sample(300, replace=False)
    elif csvf == "data/train_MK_Physics_of_The_Mind_Book.csv":
        sampled_chunks = text_chunks['text'].sample(300, replace=False)

    print("csvf = ", csvf)

    ixcounter = -1
    for txtch in tqdm(sampled_chunks, f"processing_{csvf}"):

        txtch = txtch.replace('-\n','')

        prompt = \
        f"""
        Instruction: Understand the below given input text and generate question and answer pairs.\n
        Input:\n
        {txtch}\n
        Output:\n
        """
        # print(prompt)

        # print("\n\n")

        response = llm(prompt)

        all_responsez.write(f"{response}\n")
        all_responsez.flush()

text_chunks:  964
csvf =  train_MK_FuturePhysics_Book.csv


processing_train_MK_FuturePhysics_Book.csv: 100%|██████████| 100/100 [30:16<00:00, 18.17s/it]


text_chunks:  825
csvf =  train_MK_Hyperspace_Book.csv


Streaming output truncated to the last 5000 lines.
processing_train_MK_Hyperspace_Book.csv: 100%|██████████| 825/825 [4:07:43<00:00, 18.02s/it]


text_chunks:  768
csvf =  train_MK_Physics_of_The_Impossible_Book.csv


Streaming output truncated to the last 5000 lines.
processing_train_MK_Physics_of_The_Impossible_Book.csv:  41%|████      | 123/300 [39:19<1:04:19, 21.80s/it]WARNING:ctransformers:Number of tokens (513) exceeded maximum context length (512).
processing_train_MK_Physics_of_The_Impossible_Book.csv:  42%|████▏     | 125/300 [39:52<55:14, 18.94s/it]  WARNING:ctransformers:Number of tokens (513) exceeded maximum context length (512).
processing_train_MK_Physics_of_The_Impossible_Book.csv:  43%|████▎     | 130/300 [41:24<51:17, 18.11s/it]WARNING:ctransformers:Number of tokens (513) exceeded maximum context length (512).
processing_train_MK_Physics_of_The_Impossible_Book.csv:  44%|████▎     | 131/300 [41:44<52:26, 18.62s/it]WARNING:ctransformers:Number of tokens (513) exceeded maximum context length (512).
processing_train_MK_Physics_of_The_Impossible_Book.csv:  44%|████▍     | 133/300 [42:20<50:59, 18.32s/it]WARNING:ctransformers:Number of tokens (513) exceeded maximum context length (512).


text_chunks:  1053
csvf =  train_MK_Physics_of_The_Mind_Book.csv


processing_train_MK_Physics_of_The_Mind_Book.csv:   0%|          | 1/300 [00:15<1:14:49, 15.01s/it]WARNING:ctransformers:Number of tokens (513) exceeded maximum context length (512).
processing_train_MK_Physics_of_The_Mind_Book.csv:   3%|▎         | 8/300 [02:04<1:09:45, 14.33s/it]WARNING:ctransformers:Number of tokens (513) exceeded maximum context length (512).
processing_train_MK_Physics_of_The_Mind_Book.csv:   6%|▌         | 18/300 [04:18<59:07, 12.58s/it]  WARNING:ctransformers:Number of tokens (513) exceeded maximum context length (512).
processing_train_MK_Physics_of_The_Mind_Book.csv:   8%|▊         | 25/300 [06:07<1:04:04, 13.98s/it]WARNING:ctransformers:Number of tokens (513) exceeded maximum context length (512).
processing_train_MK_Physics_of_The_Mind_Book.csv:  11%|█▏        | 34/300 [08:14<1:10:26, 15.89s/it]WARNING:ctransformers:Number of tokens (513) exceeded maximum context length (512).
processing_train_MK_Physics_of_The_Mind_Book.csv:  12%|█▏        | 37/300 [08:56<1

In [57]:
! which python3

/usr/bin/python3


In [ ]:
all_responsez.write(f"{prompt}\n")


In [ ]:
test_chunks["text"][0]

'All rights reserved. Published in the United States by Doubleday, a division of Random House, Inc., New York, and in Canada by Random House of Canada Limited, Toronto. Copyright © 2011 by Michio Kaku www.doubleday.com DOUBLEDAY and the DD colophon are registered trademarks of Random House, Inc. \x0cthis page constitutes an extension of this copyright page. LIBRARY OF CONGRESS CATALOGING-IN-PUBLICATION DATA Physics of the future : how science will shape human destiny and our daily lives by the year 2100 Michio Kaku.—1st ed.font> Kaku, Michio. Includes bibliographical references. p. cm. 1. Science—Social aspects—Forecasting. 2. Science—History—21st century. I. Title. Q175.5.K257 2011 303.4830112—dc22 2010026569 eISBN: 978-0-385-53081-1 v3.1 To my loving wife, Shizue, and my daughters, Michelle and Alyson Cover Title Page Copyright Dedication ACKNOWLEDGMENTS INTRODUCTION: Predicting the Next 100 Years FUTURE OF THE COMPUTER: Mind over Matter FUTURE OF AI: Rise of the Machines \x0cFUTURE 

In [ ]:

prompt

'\nInstruction: Understand the below given input text and generate question and answer pairs.\n\nInput:\n\n\nIn 1863, the great novelist Jules Verne undertook perhaps his most ambitious project. He wrote a prophetic novel,\ncalled Paris in the Twentieth Century, in which he applied the full power of his enormous talents to forecast the\ncoming century. Unfortunately, the manuscript was lost in the mist of time, until his great-grandson accidentally\nstumbled upon it lying in a safe where it had been carefully locked away for almost 130 years. Realizing what a\ntreasure he had found, he arranged to have it published in 1994, and it became a best seller.\n\n\nOutput:\n\n'

In [ ]:
txtch = """
In 1863, the great novelist Jules Verne undertook perhaps his most ambitious project. He wrote a prophetic novel,
called Paris in the Twentieth Century, in which he applied the full power of his enormous talents to forecast the
coming century. Unfortunately, the manuscript was lost in the mist of time, until his great-grandson accidentally
stumbled upon it lying in a safe where it had been carefully locked away for almost 130 years. Realizing what a
treasure he had found, he arranged to have it published in 1994, and it became a best seller.
"""

prompt = \
f"""
Instruction: Understand the below given input text and generate question and answer pairs.\n
Input:\n
{txtch}\n
Output:\n
"""

In [ ]:
%%time
response = llm(prompt)
print(response)

Question: When did Jules Verne start writing "Paris in the Twentieth Century"?
Answer: In 1863.

Question: For how many years was the manuscript of "Paris in the Twentieth Century" locked away safely?
Answer: For almost 130 years.
CPU times: user 53.2 s, sys: 2.51 s, total: 55.7 s
Wall time: 9.38 s
